In [2]:
import torch
from torch import nn
from d2l import torch as d2l
import tool_package


In [3]:

# 实现一下LeNet. 
# 首先我们定义一下Reshape方法。
class Reshape(torch.nn.Module):
    def forward(self, X: nn.Module):
        return X.view(-1, 1, 28, 28)



In [4]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(), # LeNet使用Sigmoid()函数. 
    nn.AvgPool2d(kernel_size=2, stride=2), # 不重叠在一起. 
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    # 做全连接层, 将输入进行平坦. 
    nn.Flatten(), # 做全连接层, 我们需要flatten. 形成一个二维, 好像全连接层, 矩阵乘积
    # 开始全连接层: 
    nn.Linear(16 * 5 * 5 ,120), nn.Sigmoid(), # 输入需要算一下的, 也就是最后一层. '
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10)
)

In [5]:
print(net)

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): Sigmoid()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): Sigmoid()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=400, out_features=120, bias=True)
  (8): Sigmoid()
  (9): Linear(in_features=120, out_features=84, bias=True)
  (10): Sigmoid()
  (11): Linear(in_features=84, out_features=10, bias=True)
)


In [10]:
# 测试输出形状: 
X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
print("source shape\t\t", X.shape)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, "output shape \t", X.shape)

source shape		 torch.Size([1, 1, 28, 28])
Conv2d output shape 	 torch.Size([1, 6, 28, 28])
Sigmoid output shape 	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape 	 torch.Size([1, 6, 14, 14])
Conv2d output shape 	 torch.Size([1, 16, 10, 10])
Sigmoid output shape 	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape 	 torch.Size([1, 16, 5, 5])
Flatten output shape 	 torch.Size([1, 400])
Linear output shape 	 torch.Size([1, 120])
Sigmoid output shape 	 torch.Size([1, 120])
Linear output shape 	 torch.Size([1, 84])
Sigmoid output shape 	 torch.Size([1, 84])
Linear output shape 	 torch.Size([1, 10])


In [7]:
"""
那么, 模型我们可以看到的思路:
就是卷积是将输入变小变小, 然后变宽变宽, 变小是指我们的关注的细节变小, 变宽, 是指我们关注的空间pattern(模式)
每一个通道就是没一个空间的pattern. 所以通道是一直在增加的, 而高宽是一直在减小的. 
不断地将空间信息压缩压缩变小,然后通道变多, 就是将压缩的信息抽取出来, 形成一种空间方式, 然后放到不同的通道中去识别.
然后MLP就是将所有这些信息拿出来, 最终通过全连接层很smooth(平滑)的将模型压缩到我们想要的输出. 

"""

'\n那么, 模型我们可以看到的思路:\n就是卷积是将输入变小变小, 然后变宽变宽, 变小是指我们的关注的细节变小, 变宽, 是指我们关注的空间pattern(模式)\n每一个通道就是没一个空间的pattern. 所以通道是一直在增加的, 而高宽是一直在减小的. \n不断地将空间信息压缩压缩变小,然后通道变多, 就是将压缩的信息抽取出来, 形成一种空间方式, 然后放到不同的通道中去识别.\n然后MLP就是将所有这些信息拿出来, 最终通过全连接层很smooth(平滑)的将模型压缩到我们想要的输出. \n\n'

In [8]:
# LeNET在Fashion-MNIST数据集中的表现. 
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size) # loda data, 加载数据. 


In [9]:
# 我们要使用GPU了. 
# 使用GPU进行评估. 

def evaluate_accuracy_gpu(net, data_iter, device=None): # 使用GPU进行评估
    """use GPU to calcuate accuracy in test datasets"""
    if isinstance(net, nn.Module): # 如果是nn.Module实现的,
        # start module evaluate
        net.eval()
        if not device:
            # if device is None. then use first parameter.device as all device.
            device = next(iter(net.parameters())).device 

    # 生成累加器: 
    metric = d2l.Accumulator(3) # generate len is 3, Accumlator. 
    
    # 评估: 
    with torch.no_grad():
        for X, y in data_iter:
            # 数据迁移. 
            if isinstance(X, list):
                # BERT微调所需. 后面介绍?
                X = [x.to(device) for x in X] 
            else:
                X = X.to(device)
            y = y.to(device)

            